In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd

# Function to fetch S&P 500 component stocks
def fetch_sp500_symbols():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    symbols = df['Symbol'].tolist()
    return symbols

# Function to screen stocks based on criteria
def stock_screener(min_pe_ratio, min_market_cap):
    sp500_symbols = fetch_sp500_symbols()

    filtered_stocks = []

    for symbol in sp500_symbols:
        stock = yf.Ticker(symbol)
        try:
            info = stock.info
            if 'quoteType' in info and info['quoteType'] == 'EQUITY':
                if 'trailingPE' in info and 'marketCap' in info:
                    pe_ratio = info['trailingPE']
                    market_cap = info['marketCap']
                    if pe_ratio is not None and market_cap is not None:
                        if pe_ratio >= min_pe_ratio and market_cap >= min_market_cap:
                            filtered_stocks.append((symbol, pe_ratio, market_cap))
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    return filtered_stocks

# Function to get user input for filtering criteria
def get_user_input():
    min_pe_ratio = float(input("Enter minimum P/E ratio: "))
    min_market_cap = float(input("Enter minimum market capitalization ($): "))
    return min_pe_ratio, min_market_cap


In [ ]:
# Run the stock screener based on user input
def run_stock_screener():
    min_pe_ratio, min_market_cap = get_user_input()
    file_name = f"filtered_stocks_PE{min_pe_ratio}_MC{min_market_cap}.csv"
    result = stock_screener(min_pe_ratio, min_market_cap)

    # Display the filtered stocks
    if result:
        df = pd.DataFrame(result, columns=['Symbol', 'P/E Ratio', 'Market Cap'])
        df.to_csv(file_name, index=False)
        print(f"Filtered stocks saved to '{file_name}'")
    else:
        print("No stocks found based on the criteria.")

# Run the stock screener based on user input
run_stock_screener()
